In [8]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.6.4")

In [11]:
import kotlinx.coroutines.*
import kotlinx.coroutines.channels.*

runBlocking {
    val channel = Channel<Int>()
    launch {
        // this might be heavy CPU-consuming computation or async logic, we'll just send five squares
        for (x in 1..5) {
            delay(400)
            channel.send(2 * x) // 짝수
        }
        println("sending even number done")
    }
    launch {
        // this might be heavy CPU-consuming computation or async logic, we'll just send five squares
        for (x in 1..5) {
            delay(800)
            channel.send(2 * x - 1) // 홀수
        }
        println("sending odd number done")
    }
    // here we print five received integers:
    for(x in 1..10) { println(channel.receive()) }
    println("Done!")
}

2
1
4
6
3
8
sending even number done
10
5
7
sending odd number done
9
Done!
